In [ ]:
# yfinance 패키지 설치
!pip install yfinance --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
sns.set()

In [ ]:
# SPY 가격 데이터 다운로드
etf = yf.Ticker('SPY')
data = etf.history(start='2000-01-01', actions=False)

# 거래량 데이터 제거
data.drop(['Volume'], inplace=True, axis=1)

data

In [ ]:
# 패러미터
param = 20

### Parkinson (1980) 변동성 추정

In [ ]:
# Parkinson 변동성 계산
vol_p = np.sqrt(252 / (4 * param * np.log(2)) * pd.DataFrame.rolling(np.log(data['High'] / data['Low']) ** 2, 20).sum())

In [ ]:
# 결과 출력
vol_p.plot()
plt.title('Parkinson Volatility')

### Garman-Klass (1980) 변동성 추정

In [ ]:
# 첫번째 항
term1 = 0.5 * pd.DataFrame.rolling(np.log(data['High'] / data['Low']) ** 2, param).sum()


# 두번째 항
term2 = (2 * np.log(2) - 1) * pd.DataFrame.rolling(np.log(data['Close'] / data['Open']) ** 2, param).sum()

# Garman-Klass 변동성 계산
vol_gk = np.sqrt((252 / param) * (term1 - term2))


In [ ]:
# 결과 출력
vol_gk.plot()
plt.title('Garman-Klass Volatility')

### Rogers-Satchell (1991) 변동성 추정

In [ ]:
# 첫번째 항
term1 = np.log(data['High'] / data['Close']) * np.log(data['High'] / data['Open']) + np.log(data['Low'] / data['Close']) * np.log(data['Low'] / data['Open'])

# Rogers-Satchell 변동성 계산
vol_rs = np.sqrt((252 / param) * pd.DataFrame.rolling(term1, param).sum())

In [ ]:
# 결과 출력
vol_rs.plot()
plt.title('Rogers-Satchell Volatility')

### Yang-Zhang (2000) 변동성 추정

In [ ]:
# 패러미터
alpha = 1.34
kappa = (alpha - 1) / (alpha + ((param + 1) / (param - 1)))

# 오버나잇 변동성
vol_o = np.log(data['Open'] / data['Close'].shift())

# 장중 변동성
vol_c = np.log(data['Close'] / data['Open'])

# Yang-Zhang 변동성 계산
vol_yz = np.sqrt(vol_o ** 2 + kappa * vol_c ** 2 + (1 - kappa) * vol_rs ** 2)

In [ ]:
# 그래프 출력
vol_yz.plot()
plt.title('Yang-Zhang Volatility')

### 모델 비교

In [ ]:
# 그래프 시각화 1
plt.plot(vol_p)
plt.plot(vol_gk)
plt.plot(vol_rs)
plt.plot(vol_yz)

plt.title('Summary of Volatility Estimation')
plt.xlabel('Year')
plt.ylabel('Volatility')

In [ ]:
# 그래프 시각화 2
plt.figure(figsize=(10, 8))
plt.plot(vol_p.iloc[-250:], label='Parkinson')
plt.plot(vol_gk.iloc[-250:], label='Garman-Klass')
plt.plot(vol_rs.iloc[-250:], label='Rogers-Satchell')
plt.plot(vol_yz.iloc[-250:], label='Yang-Zhang')

plt.title('Summary of Volatility Estimation')
plt.xlabel('Year')
plt.ylabel('Volatility')
plt.legend(loc='upper left')

Copyright 2022. 퀀트대디. All rights reserved.